In [1]:
from langchain_groq import ChatGroq
import os

os.environ["groq_api_key"] = "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"

llm = ChatGroq(
    model = "llama-3.3-70b-versatile",
    temperature=0,
    api_key= os.getenv("groq_api_key"),
)

result = llm.invoke("who is the prime minister of india")
print(result.content)

The current Prime Minister of India is Narendra Modi. He has been serving as the Prime Minister of India since May 26, 2014, and is currently in his second term, which began on May 30, 2019.


In [6]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    "https://careers.nike.com/nike-inc-nxt-space-design-innovation-graduate-internship/job/R-53145",
    header_template={"User-Agent":"Mozilla/5.0"}
    )
page_data = loader.load().pop().page_content
print(page_data)






















NIKE, Inc NXT SPACE Design Innovation Graduate Internship










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return 

In [21]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the carrer's page of a website.
    You job is to extract the job postings and return them in JSON format containing the 
    following keys: 'role','experience','skills' and 'description'.
    Only return the valid JSON , not write preamble on the top with commenting and skills sections should be in list.
    ### VALID JSON WITH NO PREAMBLE :
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke({'page_data':page_data})
print(res.content)

```json
{
  "role": "NXT SPACE Design Innovation Graduate Internship",
  "experience": "1-3 years",
  "skills": [
    "Industrial Design",
    "Product Design",
    "Fashion Design",
    "Materials Design",
    "Illustrator",
    "Photoshop",
    "Miro",
    "Keynote",
    "3D design tools"
  ],
  "description": "We are looking for an exceptional Product Design Innovator and entrepreneurial problem solver that is driven by curiosity, leads with optimism and prefers collaboration over competition. You will be working as an intern on the NXT SPACE Design Innovation Team within the Nike Explore Team (NXT) to create future forward, original innovative designs and concepts, including but not limited to footwear, apparel and experiences that reflect and push the boundaries of Nike's design philosophy and standards."
}
```


In [22]:
from langchain_core.output_parsers import JsonOutputParser 

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'NXT SPACE Design Innovation Graduate Internship',
 'experience': '1-3 years',
 'skills': ['Industrial Design',
  'Product Design',
  'Fashion Design',
  'Materials Design',
  'Illustrator',
  'Photoshop',
  'Miro',
  'Keynote',
  '3D design tools'],
 'description': "We are looking for an exceptional Product Design Innovator and entrepreneurial problem solver that is driven by curiosity, leads with optimism and prefers collaboration over competition. You will be working as an intern on the NXT SPACE Design Innovation Team within the Nike Explore Team (NXT) to create future forward, original innovative designs and concepts, including but not limited to footwear, apparel and experiences that reflect and push the boundaries of Nike's design philosophy and standards."}

In [23]:
type(json_res)

dict

In [24]:
import pandas as pd 
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [26]:
import chromadb
import uuid

client = chromadb.PersistentClient()
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=row["Techstack"],
            metadatas={"links":row["Links"]},
            ids=[str(uuid.uuid4())]
        )


In [27]:
links = collection.query(query_texts=["Experience in Python","Expertise in React Native"],n_results=2).get("metadatas")

In [28]:
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [31]:
job= json_res
job["skills"]

['Industrial Design',
 'Product Design',
 'Fashion Design',
 'Materials Design',
 'Illustrator',
 'Photoshop',
 'Miro',
 'Keynote',
 '3D design tools']

In [33]:
links=collection.query(query_texts=job["skills"],n_results=2).get("metadatas")
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [35]:
from langchain.prompts import PromptTemplate

prompt_email = PromptTemplate.from_template(
    """ 
    ### JOB DESCRIPTION:
    {job_description}
    ### INSTRUCTION:
    You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
    Remember you are Mohan, BDE at AtliQ. 
    Do not provide a preamble.
    ### VALID JSON WITH NOT PREAMBLE:"""
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description":str(job),"link_list":links})
print(res.content)

Subject: Empowering Innovation: AtliQ's Expertise for NXT SPACE Design Innovation Graduate Internship

Dear Hiring Manager,

I came across the NXT SPACE Design Innovation Graduate Internship role at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company dedicated to facilitating seamless integration of business processes through automated tools. Our expertise in designing and developing innovative solutions can support the NXT SPACE Design Innovation Team in creating future-forward designs and concepts.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team's capabilities in Industrial Design, Product Design, and 3D design tools can complement the skills required for this internship. We can help streamline design processes, enhance collaboration, and drive innovation using tools like Illustrator, Photoshop, Miro, and Keynot